<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i try to explore different dataset to look if they can be connected, linked or somehow can talk to eachother.**
*   *Data by Frederik*
*   *Frida dataset of Nutrient content of food items*
*   *Reciept from my personal Storebox account*
*   *link from TradeSync data from GS1*
*   *Climate Data from Concito*

In [36]:
#!pip install PyPDF2

In [35]:
# Necessary libaries
import pandas as pd
import os
import PyPDF2

# Data by Frederik

***Columns description***

*   **itemname:** name provided by the merchant. It often, but not always, contains a enough information to ID the product.  Sometimes weight, or other information is also provided here.
*   **itemnumber:** the product itemnumber. It can be a standardized GTIN 13 number ( 12-13 digits) supported by GS1 or a non standard number created by the retailer to identify i.e. Bananas
*   **category:** a category code not sure how to interpret and often missing
*   **percent_purchases:** Percent off all receipt lines in the dataset where each line is a purchas

In [3]:
# Import data
path_iss_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/cpd_summary.csv"

# Open data as csv file
data_iss= pd.read_csv(path_iss_data, sep= ";", index_col=False, encoding='latin-1')

# lets see frist 3 and last 3 rows od the dataset
data_iss.head(3).append(data_iss.tail(5))

,itemname,itemnumber,category,percent_purchases
0,PANT,8880171,845,"3,369"
1,PANT,8880171,NaN,"2,055"
2,PANT,8880172,845,"1,497"
112761,PINOTEX SUPERDEC SOR,61009053404,6100,"-0,002"
112762,QUICK GEVINSTER,214196,926,"-0,002"
112763,UKRUDTSDUG BASIC 1X,42139019916,4213,"-0,002"
112764,TRAPPEVANGE,5708832546219,600,"-0,004"
112765,BR MÆRKER RETUR,5766183013538,21,"-0,005"


In [4]:
# Only print one selective row of interest to deep deeper
#data_iss[data_iss.itemname == "BANANER, LØSE" or "ØKO BANANER OR LØSE" or "BANANER"] # or "ØKO BANANER, LØSE", HAVFRISK LAK, APPELSINER 2 KG,Agurk

# only some of the rows of different kinds of the banana
data_iss.iloc[[11, 12, 14, 18], :]   

,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
12,"BANANER,LØSE",5711044628659,NaN,"0,565"
14,BANANER,406,0000000030,"0,419"
18,"BANANER,LØSE",5711044628659,89,"0,266"


***In above ddataset, only one food itemname (fx `"ØKO BANANER, LØSE"`) is appear in different rows with;***
*   Many different itemnumber, even some are 13 digits and other are 8 digits
*   NULL percent purchage

In [ ]:
# everytime refrence this cell,new dataframewill be displayed
data_iss.sample(1)

***Here i like to use NLP to find split itemname to name and other entity***

In [6]:
#!pip install nltk

In [7]:

# Liaries for NLP
from nltk.tokenize import word_tokenize

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("omw-1.4")

stop_words= set(stopwords.words("danish"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [8]:
# here i am going to use Regular Express (RegEx) for turning sentance to each word (Word Tokenization)
# library for regular expression
import re

def remove_delimiters(sentence):
  return re.findall(r'\w+', sentence)

sents= " UKRUDTSDUG BASIC 1X  " # sdujh	ØKO BANANER, LØSE jhghujw, hsdfgw"

trimmed_sent= remove_delimiters(sents)
trimmed_sent

['UKRUDTSDUG', 'BASIC', '1X']

In [9]:
"""#WWord tagging
words= nltk.word_tokenize(sents)
pos_tag= nltk.pos_tag(words)

words
pos_tag"""

'#WWord tagging\nwords= nltk.word_tokenize(sents)\npos_tag= nltk.pos_tag(words)\n\nwords\npos_tag'

# FRIDA Food Composition Database
***FRIDA Food Composition Database  contain mainly Nutrition value of the each food items***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

In [12]:
# Import FRIDA Nutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=None, header=0, skiprows=0).round(decimals = 2)
data_frida.head(6)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Svind,"Energi, kJ","Energi, kcal",Nitrogen-til-protein faktor,"Nitrogen, total","Protein, videnskabelig","Protein, deklaration",...,Arginin,Histidin,Alanin,Asparaginsyre,Glutaminsyre,Glycin,Prolin,Hydroxyprolin,Serin,Massefylde
0,Nummer,gruppe,Navn,%,kJ,kcal,-,g,g,g,...,mg,mg,mg,mg,mg,mg,mg,g,mg,g/l
1,1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,...,40,12,31,96,90,25,23,iv,30,iv
2,2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,...,6,4,11,78,22,8,8,iv,11,iv
3,3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,...,49,84,35,117,106,37,38,iv,53,iv
4,4,Rod- og knoldgrøntsager,"Kartoffel, uspec., rå",25,326,77,6.25,0.324,2,2,...,84,36,71,428,269,58,71,iv,71,iv
5,5,Bærfrugt,"Hindbær, rå",0,215,51,6.25,0.219,1.4,1.4,...,iv,iv,iv,iv,iv,iv,iv,iv,iv,iv


In [13]:
# Import FRIDA Nutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=None, header=None, skiprows=2).round(decimals = 2).iloc[:, :26]

# Lets define and Assign columns name that defined just above
column_names= ['FoodId', 'Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']
data_frida.columns= column_names

# lets see frist 3 and last 3 rows od the dataset
data_frida.head(3).append(data_frida.tail(3))

#data_frida[data_frida.Gruppe == 'Sydfrugt'] #"Frugt"-grøntsager, Sydfrugt, Bærfrugt
#data_frida[data_frida.Navn == "Banan, rå"]


,FoodId,Gruppe,Navn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
1183,1728,Fedtrig frugt og frø,Chiafrø,iv,1837,447,iv,3.425,21.4,21.4,...,0.887,iv,4.4,93.9,6.1,0,iv,0,0,0
1184,1729,Kornsorter,Perlespelt,iv,1386,329,iv,1.94,12.1,12.1,...,0.598,iv,1.5,88.8,11.2,0,iv,0,0,0
1185,1730,Kornsorter,Perlebyg,iv,1413,335,iv,1.455,9.1,9.1,...,0.548,iv,1.3,89.2,10.8,0,iv,0,0,0


***Lets read the data from another sheet of the Excel file of FRIDA data***

---



In [14]:
# Import FRIDA Nutrient data
path_frida_data_groups="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as pandas dataframe with adjusted column name
data_frida_groups= pd.read_excel(path_frida_data_groups, sheet_name=3, index_col=None, header=0, skiprows=0)[["FoodGroupID",	"GroupID",	"GroupNameDK",	"GroupNameUK",	"Hieraky",	"ID", "ParantFoodGroupId"]]
data_frida_groups.head(3)

,FoodGroupID,GroupID,GroupNameDK,GroupNameUK,Hieraky,ID,ParantFoodGroupId
0,1,1,LST-GRUPPER,DANSDA-GROUPS,1,1,1
1,2,1,Konsummælk og mælkekonserves,Milk and milk preserves,2,70,1
2,3,1,Andre mælkeprodukter,Other milk products,3,81,2


In [15]:
# lets have a look row with  "Banan, rå" to have understanding of the rows
data_frida_groups[data_frida_groups.ParantFoodGroupId == 47]

,FoodGroupID,GroupID,GroupNameDK,GroupNameUK,Hieraky,ID,ParantFoodGroupId
39,48,1,Anden frugt,Other fruit,3,101,47
40,49,1,Kernefrugt,Pome fruit,3,102,47
41,50,1,Stenfrugt,Stone fruit,3,104,47
42,51,1,Bærfrugt,Soft fruit,3,105,47
43,52,1,Sydfrugt,Tropical or subtropical fruit,3,106,47
44,53,1,Fedtrig frugt og frø,High-fat fruit and seeds,3,107,47
45,54,1,Frugtsaft og juice,Fruit juices,3,108,47
46,55,1,Frugtprodukter,Fruit products,3,109,47


***The FoodGroupId of the Sydfrugt(Tropical or subtropical fruit) category is 52.In addition, ParantFoodGroupId will give the Top category name that GroupNameDK belong to! fx.47 means "Frugt og frugtprodukter"***

# Now lets open another file of  FRIDA dataset: Additional Link Data

***Lets read the data from***
---

In [23]:
# Import FRIDA AdditionalLink data
path_frida_aditional_link="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaAdditionalLinkData.xlsx"

# Lets open with little modificaton so easy to read and understand
data_frida_aditional_link= pd.read_excel(path_frida_aditional_link, sheet_name=3, index_col=None, header=0, skiprows=0).round(decimals = 2)

# lets see first 3 and last 3 rows od the dataset
data_frida_aditional_link.head(3).append(data_frida_aditional_link.tail(3))

#data_frida[data_frida.Gruppe == 'Sydfrugt'] #"Frugt"-grøntsager, Sydfrugt, Bærfrugt
#data_frida_aditional_link[data_frida_aditional_link.FødevareNavn == "Banan, rå"]

,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51,Bærfrugt,Soft fruit
1,"Æble, uspec., råt","Apple, raw, all varieties",2,Malus domestica Borkh.,A01DJ#F02.A067D$F26.A07XD,49,Kernefrugt,Pome fruit
2,"Banan, rå","Banana, raw",3,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52,Sydfrugt,Tropical or subtropical fruit
1246,"Stykker, med mycoprotein","Pieces, with mycoprotein",1793,NaN,NaN,169,Plantebaserede produkter,Plant-based products
1247,"Pølse, med sojaprotein","Sausage, with soy protein",1794,NaN,NaN,169,Plantebaserede produkter,Plant-based products
1248,"Oksekød, filet, marmoreret, rå","Beef, entrecote/rib eye, raw",1795,Bos taurus,NaN,58,Oksekød,Beef


***This dataset have some of the very intresting feactures.***

*   With the column "FoodId", we can connnecet wiith other dataset. 
*   "FoodEx2Code" column could be usefull to connect other dataset of XXXX
*   "FoodGroupID" column will be usefull to define food category
*   "TaxonomicName" column will be usefull to define Scientific nameof the food item

***Combining and visualizing information***

In [17]:
# lets acess the row of the iss_data
#sample_data= data_iss.sample(1)
sample_data_iss= data_iss[data_iss.itemname == "ØKO BANANER, LØSE"] 

# Acess only the value of itenname and itemnumber
item_name= sample_data_iss.itemname.values[0]
item_number= sample_data_iss.itemnumber.values[0]

print("*" *40)
print("From iss_data")
display(sample_data_iss)
print("\nitem_name: ", item_name, "\nitem_number: ", item_number)
print("*" *40)

# Lets acess from FRIDA Nutrient data
#data_frida[data_frida.Gruppe == 'Sydfrugt'] #"Frugt"-grøntsager, Sydfrugt, Bærfrugt
sample_data_frida= data_frida[data_frida.Navn == "Banan, rå"]
food_name= sample_data_frida.Navn.values[0]
food_id= sample_data_frida.FoodId.values[0]
food_group= sample_data_frida.Gruppe.values[0]

print("*" *40)
print("From FridaNutrient data")
display(sample_data_frida)
print("\nfood_name: ", food_name, "\nfood_id: ",food_id, "\nfood_group: ", food_group)
print("*" *40)

# Lets acess from FRIDA Nutrient data with group
sample_data_frida_groups= data_frida_groups[data_frida_groups.GroupNameDK == food_group].iloc[:, [0, 2,6]]   

food_group= sample_data_frida_groups.GroupNameDK.values[0]
Food_GroupID= sample_data_frida_groups.FoodGroupID.values[0]
Parant_Food_GroupId= sample_data_frida_groups.ParantFoodGroupId.values[0]

print("*" *40)
print("From FridaNutrient GroupData")
display(sample_data_frida_groups)
print("\nfood_group: ", food_group, "\nFood_GroupID: ",Food_GroupID, "\nParant_Food_GroupId: ", Parant_Food_GroupId)
print("*" *40)

# Lets acess from FRIDA Nutrient data with additional link
sample_data_frida_aditionlink= data_frida_aditional_link[data_frida_aditional_link.FødevareNavn == food_name].iloc[:, [0, 2, 3,4,5,6]]  

Food_name= sample_data_frida_aditionlink.FødevareNavn.values[0]
Food_ID= sample_data_frida_aditionlink.FoodID.values[0]
Food_GroupId= sample_data_frida_aditionlink.FoodGroupID.values[0]
Food_group= sample_data_frida_aditionlink.FødevareGruppe.values[0]
Food_taxonomy= sample_data_frida_aditionlink.TaxonomicName.values[0]
Food_Ex2_code = sample_data_frida_aditionlink.FoodEx2Code.values[0]

print("*" *40)
print("From FridaNutrient AditionLink Data")
display(sample_data_frida_aditionlink)
print("\nFood_name: ", Food_name, "\nFood_ID: ", Food_ID, "\nFood_GroupId: ", Food_GroupId, "\nFood_group: ", Food_group, "\nFood-taxonomy: ", Food_taxonomy, "\nFood_Ex2_code: ", Food_Ex2_code)
print("*" *40)






****************************************
From iss_data


,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
9212,"ØKO BANANER, LØSE",5712580050171,89,"0,002"
27542,"ØKO BANANER, LØSE",20008932,89,"0,001"
27543,"ØKO BANANER, LØSE",20014018,89,"0,001"
27544,"ØKO BANANER, LØSE",20023393,89,"0,001"
27545,"ØKO BANANER, LØSE",5712580000114,89,"0,001"
27546,"ØKO BANANER, LØSE",5712580024349,89,"0,001"
27547,"ØKO BANANER, LØSE",5712580034065,89,"0,001"
27548,"ØKO BANANER, LØSE",5712580251912,89,"0,001"
27549,"ØKO BANANER, LØSE",5712580317717,89,"0,001"



item_name:  ØKO BANANER, LØSE 
item_number:  5712873336944
****************************************
****************************************
From FridaNutrient data


,FoodId,Gruppe,Navn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
2,3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0



food_name:  Banan, rå 
food_id:  3 
food_group:  Sydfrugt
****************************************
****************************************
From FridaNutrient GroupData


,FoodGroupID,GroupNameDK,ParantFoodGroupId
43,52,Sydfrugt,47



food_group:  Sydfrugt 
Food_GroupID:  52 
Parant_Food_GroupId:  47
****************************************
****************************************
From FridaNutrient AditionLink Data


,FødevareNavn,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe
2,"Banan, rå",3,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52,Sydfrugt



Food_name:  Banan, rå 
Food_ID:  3 
Food_GroupId:  52 
Food_group:  Sydfrugt 
Food-taxonomy:  Musa x paradisiaca L. 
Food_Ex2_code:  A01LC#F01.A064L$F02.A067D
****************************************


***Above dataset do not contain NAN value and all of the columns are object type, so some of the columns need to be changed to interger or float data type to work as numbers value***

# Recipt from my personal Storebox account
- This data set i download from my own storebox account. Lets see what kind of information we can extract from storebox account

***The Store-box dataset contain one dataset of recieptand another dataset of the actual list of itemspurchaged each time***

In [18]:
#Import data
path_storebox_data= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/storbox_recipts.json"

#Loading dat in dataframe 
data_storebox= pd.read_json(path_storebox_data)
# ['id', 'type', 'receiptId', 'purchaseDate', 'purchaseDateTimeString', 'orderNumber', 'price', 'merchant', 'receiptLines', 'terminalId', 'payments', 'cards', 
#     'barcode', 'headerTexts', 'footerTexts', 'userIds', 'imageId', 'receiptImage', 'note', 'tags', 'headerText', 'footerText', 'purchaseDateTimeZoneOffset', 'legalReceipt']

# Lets open only columns that make sence
data_storebox= data_storebox[["id", "receiptId", "purchaseDate", "orderNumber", "price", "merchant",'barcode']]
data_storebox.head(3)

,id,receiptId,purchaseDate,orderNumber,price,merchant,barcode
0,236769820,05jrqpia3ofdzsbq773yme4on0ko1f87,1520106179000,5320020898,"{'amount': 229.29, 'vat': 45.86, 'currency': '...","{'merchantId': 'netto', 'storeId': None, 'name...","{'type': 'interleaved2of5', 'value': '07344180..."
1,323704901,0dtsbqbwkirnyxfjqwdaogf5bli877yi,1546781345000,8190010157,"{'amount': 19.15, 'vat': 3.83, 'currency': 'DK...","{'merchantId': 'netto', 'storeId': '7345', 'na...","{'type': 'interleaved2of5', 'value': '07345190..."
2,230788984,0of79b19he9g7g5wazomg1tdoc3yh0xk,1518030461000,5080021069,"{'amount': 115.22, 'vat': 23.04, 'currency': '...","{'merchantId': 'netto', 'storeId': None, 'name...","{'type': 'interleaved2of5', 'value': '07344180..."


***With the receiptId i can go to indiviisual reciept and extract list of purchage items. However the reciept is in PDF so need little advance RegEx code so i have not done it yet but will update soon.***


**Now lets  open reciept of  of the very first column (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above data**

In [37]:
# Lets open reciept of very first columns (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above dataset (data_storbox)
data_storbox_reciept= "/content/05jrqpia3ofdzsbq773yme4on0ko1f87-14736470631022876123.pdf" # "https://github.com/mrchapagain/food_ontology/raw/main/storbox_recipt_pdf.pdf"

# read information from PDF file (open it in read binary mode)
pdfFileObj = open(data_storbox_reciept, 'rb')

# call and store PdfFileReader
# object in pdfReader
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# to access first page if the PDF file have multiple pages, pass 0
pageObj = pdfReader.getPage(0)

# extract the page object by extractText() function
texts = pageObj.extractText()

# print the extracted texts
print(texts)

UTTERSLEVVEJ 11
2400 Kbh. NV
SØDMÆLK 8,50
SØDMÆLK 8,50
OLLINEO SOLSIKKE OL 9,95
CRYSLI 4 NØD 32,85
DANONINO 14,50
BACON I SKV. 2-PAK 17,90
SPINAT 12,09
5 BANANER,LØSE
A 2,00 10,00
RABAT 5,00-
RØDLØG 8,00
UDL. AGURK 5,00
HAVFRISK LAK 89,00
RABAT 20,00-
5 PÆRER, LØSE
A 2,00 10,00
APPELSINER 2 KG 18,00
GRANATÆBLE, LØS 10,00
TOTAL 229,29
KONTANT 31,50
TOTAL 197,79
DANKORT 197,79
MOMS UDGØR 45,86
Du blev betjent af:
Kevin
9 2 898 03 03 18 20:44
Butik 7344 MOMSNR.35954716
KIG FORBI WWW.NETTO.DK
OG WWW.JOB.NETTO.DK
8-22 ALLE UGENS 7 DAGE
2018-03-03 20:44
Køb DKK 197,79
_________
Dankort PSN: 00
Contactless
XXXX XXXX XXXX 4955
Term: 80471832-143669
NETS A/S
4815734
KC1 Nets no:0004815734
ATC:00292 AED:000000
AID: A0000001214711
PSAM: 5374978-0000544044
ARC:00 STATUS:0000


***I have extracted data from PDF file now first need to turn it to dictionary and then DataFrame.***

In [38]:
lines= texts.rstrip("\n")
lines

'UTTERSLEVVEJ 11\n2400 Kbh. NV\nSØDMÆLK 8,50\nSØDMÆLK 8,50\nOLLINEO SOLSIKKE OL 9,95\nCRYSLI 4 NØD 32,85\nDANONINO 14,50\nBACON I SKV. 2-PAK 17,90\nSPINAT 12,09\n5 BANANER,LØSE\nA 2,00 10,00\nRABAT 5,00-\nRØDLØG 8,00\nUDL. AGURK 5,00\nHAVFRISK LAK 89,00\nRABAT 20,00-\n5 PÆRER, LØSE\nA 2,00 10,00\nAPPELSINER 2 KG 18,00\nGRANATÆBLE, LØS 10,00\nTOTAL 229,29\nKONTANT 31,50\nTOTAL 197,79\nDANKORT 197,79\nMOMS UDGØR 45,86\nDu blev betjent af:\nKevin\n9 2 898 03 03 18 20:44\nButik 7344 MOMSNR.35954716\nKIG FORBI WWW.NETTO.DK\nOG WWW.JOB.NETTO.DK\n8-22 ALLE UGENS 7 DAGE\n2018-03-03 20:44\nKøb DKK 197,79\n_________\nDankort PSN: 00\nContactless\nXXXX XXXX XXXX 4955\nTerm: 80471832-143669\nNETS A/S\n4815734\nKC1 Nets no:0004815734\nATC:00292 AED:000000\nAID: A0000001214711\nPSAM: 5374978-0000544044\nARC:00 STATUS:0000'

# Climate Data from Concito

In [39]:
#Import data
path_climate_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/ClimateData.xlsx"

# Loading the data into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]

# Original columns name are replaced with ralavant names (col_names)
data_climate= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).round(decimals = 2)
data_climate.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/io/excel/_base.py:1292: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  **kwds,


,id,Product_dk,Category_dk,Product_en,Category_en,Unit,Agriculture,iLUC,Processing,Packaging,...,GPC_Category_en,GPC_Category_dk,GPC_Level4_dk,product_type,GPC_level1,Food_group,GPC_level2,Un/Processed,GPC_Level3,Extra_category
0,Ra00001,"Peberfrugt, rød, rå",Grøntsager,"Pepper, sweet, red, raw",Vegetables,kg,0.25,0.03,0.0,0.14,...,Vegetables,Grøntsager,10006191,Square Tapering Sweet Peppers (Peg Top),50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260400.0,Peppers
1,Ra00002,"Tomat, uspec., rå",Grøntsager,"Tomato, ripe, raw, origin unknown",Vegetables,kg,0.07,0.01,0.0,0.14,...,Vegetables,Grøntsager,10006165,Tomatoes – Round,50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260300.0,Tomatoes
2,Ra00003,"Squash, rå",Grøntsager,"Squash, raw",Vegetables,kg,0.14,0.04,0.0,0.06,...,Vegetables,Grøntsager,10006355,Squash (Calabaza),50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260700.0,Cucurbits – Edible Peel


***The reason to open this climate dataset is to see if it is possible link other dataset with column name "GPC_Level" which is releted to GS1 Trade Sync code.***

**Let´s have a look dataset with only selected columns that might make sense\**

In [40]:
cols_to_use= ["Product_en", "Category_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_level1", "GPC_level2", "GPC_Level3", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg"]

data_climate_selected= data_climate.loc[:, cols_to_use].round(decimals = 2)
data_climate_selected.head()

,Product_en,Category_en,ID_Food,ID_Pack,ID_Retail,GPC_level1,GPC_level2,GPC_Level3,GPC_Level4_dk,Fat_g,Carb_g,Protein_g,Energy_KJ,Agriculture,iLUC,Processing,Packaging,Transport,Retail,Total_CO2_eq/kg
0,"Pepper, sweet, red, raw",Vegetables,Cm039,Pa059,Ma0332,50000000.0,50260000.0,50260400.0,10006191,0.1,5.2,0.9,133.0,0.25,0.03,0.0,0.14,0.58,0.01,1.02
1,"Tomato, ripe, raw, origin unknown",Vegetables,Cm167,Pa059,Ma0332,50000000.0,50260000.0,50260300.0,10006165,0.1,3.2,0.8,87.0,0.07,0.01,0.0,0.14,0.48,0.01,0.70
2,"Squash, raw",Vegetables,Cm135,Pa040,Ma0332,50000000.0,50260000.0,50260700.0,10006355,0.2,2.4,1.4,81.0,0.14,0.04,0.0,0.06,0.58,0.01,0.83
3,"Aubergine, raw",Vegetables,Cm056,Pa059,Ma0332,50000000.0,50260000.0,50260500.0,10006128,0.1,3.1,0.9,83.0,0.16,0.02,0.0,0.14,0.65,0.01,0.97
4,"Tomato, dried",Vegetables,Fa0152,Pa066,Ma0332,50000000.0,50100000.0,50102100.0,10000272,3.0,43.5,14.1,1187.0,0.91,0.09,-0.0,0.26,0.68,0.01,1.95


# Data from COOP store (store data)

In [ ]:
path_coop_data= "https://raw.githubusercontent.com/mrchapagain/ConsumerDataAnalytics/main/storedata/storedata_csv.csv"
#data_coop= pd.read_excel(path_coop_data, sheet_name=1, header=0, skiprows=0).round(decimals = 2)

data_coop= pd.read_csv(path_coop_data, index_col=0, sep= ";", on_bad_lines='skip', engine='python')

data_coop.head()

***These information are saved wrong way; all the columns of csv in one column so it open like one column***

So i am going to open with string split method with comma " ,"

# Latest downloaded GS1 Trade-Sync dataset

In [143]:
#Import data
path_tradesync_excel= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/GS1_GPC_Dataset.xlsx" 

# Lets open data as they are
data_tradesync_excel= pd.read_excel(path_tradesync_excel, sheet_name=0, index_col=None, header=0, skiprows=0)

# lets see frist 3 and last 3 rows od the dataset
data_tradesync_excel.head(2).append(data_tradesync_excel.tail(2))

,SegmentCode,SegmentTitle,SegmentDefinition,FamilyCode,FamilyTitle,FamilyDefinition,ClassCode,ClassTitle,ClassDefinition,BrickCode,BrickTitle,BrickDefinition_Includes,BrickDefinition_Excludes,AttributeCode,AttributeTitle,AttributeDefinition,AttributeValueCode,AttributeValueTitle,AttributeValueDefinition
0,70000000,Arts/Crafts/Needlework,NaN,70010000,Arts/Crafts/Needlework Supplies,NaN,70010200,Airbrushing Supplies,NaN,10001686,Airbrushes (Powered),Includes any products that can be described/ob...,Specifically excludes Spray Paint and Aerosols...,NaN,NaN,NaN,NaN,NaN,NaN
1,70000000,Arts/Crafts/Needlework,NaN,70010000,Arts/Crafts/Needlework Supplies,NaN,70010200,Airbrushing Supplies,NaN,10001688,Airbrushing Equipment - Replacement Parts/Acce...,Includes any products that can be described/ob...,Excludes products such as complete Airbrushes.,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,"Indicates, with reference to the product brand...",30008542.0,AIRBRUSH CONTROL VALVE,NaN
183206,77000000,Vehicle,NaN,77030000,Cars and Motorcycles,NaN,77030200,Motorcycles/ All Terrain Vehicles/ Utility Tas...,NaN,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,An Utility Task Vehicle (UTV) or Recreational ...,"Excludes Motorcycles, Mopeds, Scooters, All-Te...",20002877.0,Type of Energy,"Indicates, with reference to the product brand...",30002518.0,UNIDENTIFIED,This term is used to describe those product at...
183207,77000000,Vehicle,NaN,77050000,Specialty Vehicles,NaN,77050100,Specialty Vehicles,NaN,10008050,Specialty Vehicles,Includes any vehicle that can be described/obs...,"Excludes vehicles such as land vehicles, water...",NaN,NaN,NaN,NaN,NaN,NaN


***Lets open only the selective columns that make sense***

In [104]:
# Selective columns names
columns = ['SegmentCode', 'SegmentTitle', 'FamilyCode', 'FamilyTitle', 'ClassCode', 'ClassTitle', 'BrickCode', 'BrickTitle', 'AttributeCode', 'AttributeTitle', 'AttributeValueCode', 'AttributeValueTitle']

# Open the dataset with selective columns names
data_tradesync_excel[columns].head(2).append(data_tradesync_excel[columns].tail(2))

,SegmentCode,SegmentTitle,FamilyCode,FamilyTitle,ClassCode,ClassTitle,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle
0,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001686,Airbrushes (Powered),NaN,NaN,NaN,NaN
1,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001688,Airbrushing Equipment - Replacement Parts/Acce...,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,30008542.0,AIRBRUSH CONTROL VALVE
183206,77000000,Vehicle,77030000,Cars and Motorcycles,77030200,Motorcycles/ All Terrain Vehicles/ Utility Tas...,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,20002877.0,Type of Energy,30002518.0,UNIDENTIFIED
183207,77000000,Vehicle,77050000,Specialty Vehicles,77050100,Specialty Vehicles,10008050,Specialty Vehicles,NaN,NaN,NaN,NaN


Description of columns:
1.   SegmentCode
2.   SegmentTitle
1.   SegmentDefinition
2.   FamilyCode
1.   FamilyTitle
2.   FamilyDefinition
1.   ClassCode
2.   ClassTitle
1.   ClassDefinition
2.   BrickCode
1.   BrickTitle
2.   BrickDefinition_Includes
1.   BrickDefinition_Excludes
2.   AttributeCode
1.   AttributeTitle
2.   AttributeDefinition
1.   AttributeValueCode
2.   AttributeValueTitle
1.   AttributeValueDefinition

In [144]:
data_tradesync_excel[data_tradesync_excel.BrickTitle == "Apples"] # Brickcode:10005897, classcode: 50250700

,SegmentCode,SegmentTitle,SegmentDefinition,FamilyCode,FamilyTitle,FamilyDefinition,ClassCode,ClassTitle,ClassDefinition,BrickCode,BrickTitle,BrickDefinition_Includes,BrickDefinition_Excludes,AttributeCode,AttributeTitle,AttributeDefinition,AttributeValueCode,AttributeValueTitle,AttributeValueDefinition
68644,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002794.0,Apple Variety,"Indicates, with reference to the product brand...",30017391.0,ADAM'S PEARMAIN,NaN
68645,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002794.0,Apple Variety,"Indicates, with reference to the product brand...",30017392.0,ADMIRAL,NaN
68646,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002794.0,Apple Variety,"Indicates, with reference to the product brand...",30014995.0,AFRICAN RED,NaN
68647,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002794.0,Apple Variety,"Indicates, with reference to the product brand...",30017393.0,AHRA,NaN
68648,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002794.0,Apple Variety,"Indicates, with reference to the product brand...",30017394.0,AHRISTA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69503,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002737.0,Quality (UNECE Standard),"Indicates, with reference to the product brand...",30014956.0,"""EXTRA"" CLASS",Products in this class must be of superior qua...
69504,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002737.0,Quality (UNECE Standard),"Indicates, with reference to the product brand...",30014608.0,CLASS I,Products in this class must be of good quality...
69505,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002737.0,Quality (UNECE Standard),"Indicates, with reference to the product brand...",30014609.0,CLASS II,This class includes products that do not quali...
69506,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50250800,Pome Fruits,NaN,10005900,Apples,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20002737.0,Quality (UNECE Standard),"Indicates, with reference to the product brand...",30002515.0,UNCLASSIFIED,This term is used to describe those product at...


In [142]:
# Uniques values from the columns
data_tradesync_excel.SegmentCode.unique()

array([70000000, 68000000, 53000000, 83000000, 74000000, 47000000,
       67000000, 66000000, 65000000, 94000000, 58000000, 78000000,
       50000000, 63000000, 87000000, 51000000, 72000000, 93000000,
       75000000, 73000000, 81000000, 89000000, 88000000, 96000000,
       61000000, 64000000, 10000000, 79000000, 99000000, 98000000,
       85000000, 91000000, 95000000, 71000000, 62000000, 92000000,
       60000000, 84000000, 80000000, 82000000, 86000000, 77000000])

In [83]:
data_tradesync_excel[data_tradesync_excel.AttributeCode == 20000743] # Brickcode:10005897, classcode: 50250700

,SegmentCode,SegmentTitle,SegmentDefinition,FamilyCode,FamilyTitle,FamilyDefinition,ClassCode,ClassTitle,ClassDefinition,BrickCode,BrickTitle,BrickDefinition_Includes,BrickDefinition_Excludes,AttributeCode,AttributeTitle,AttributeDefinition,AttributeValueCode,AttributeValueTitle,AttributeValueDefinition
42382,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50251300,Annona,NaN,10006433,Annona Other,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30014610.0,AFGHANISTAN,NaN
42383,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50251300,Annona,NaN,10006433,Annona Other,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30016053.0,ÅLAND ISLANDS,NaN
42384,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50251300,Annona,NaN,10006433,Annona Other,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30014612.0,ALBANIA,NaN
42385,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50251300,Annona,NaN,10006433,Annona Other,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30014613.0,ALGERIA,NaN
42386,50000000,Food/Beverage/Tobacco,NaN,50250000,Fruits - Unprepared/Unprocessed (Fresh),NaN,50251300,Annona,NaN,10006433,Annona Other,Includes any product that can be described/obs...,Specifically excludes: Prepared/Processed or U...,20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30014614.0,AMERICAN SAMOA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174744,96000000,Monetary Assets,NaN,96010000,Currency/Postage/Certificates,NaN,96010100,Currency/Certificates,Includes products that represent a medium of e...,10007969,Bills,Includes any products that can be described as...,"Excludes: Bank Notes/Cheques, Coins and Digita...",20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30002515.0,UNCLASSIFIED,This term is used to describe those product at...
174745,96000000,Monetary Assets,NaN,96010000,Currency/Postage/Certificates,NaN,96010100,Currency/Certificates,Includes products that represent a medium of e...,10007969,Bills,Includes any products that can be described as...,"Excludes: Bank Notes/Cheques, Coins and Digita...",20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30002518.0,UNIDENTIFIED,This term is used to describe those product at...
174746,96000000,Monetary Assets,NaN,96010000,Currency/Postage/Certificates,NaN,96010100,Currency/Certificates,Includes products that represent a medium of e...,10007971,Coins,Includes any products that can be described as...,"Excludes: Bills, Bank Notes/Cheques and Digita...",20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30015349.0,EUROPEAN UNION,NaN
174747,96000000,Monetary Assets,NaN,96010000,Currency/Postage/Certificates,NaN,96010100,Currency/Certificates,Includes products that represent a medium of e...,10007971,Coins,Includes any products that can be described as...,"Excludes: Bills, Bank Notes/Cheques and Digita...",20000743.0,Country/Zone of Origin,"Indicates, with reference to the product brand...",30002515.0,UNCLASSIFIED,This term is used to describe those product at...


In [103]:
data_tradesync_excel.SegmentCode

0         70000000
1         70000000
2         70000000
3         70000000
4         70000000
            ...   
183203    77000000
183204    77000000
183205    77000000
183206    77000000
183207    77000000
Name: SegmentCode, Length: 183208, dtype: int64

# TradeSync data from GS1 

In [41]:
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [50]:
#Import data
path_tradesync_data= "/content/Message-000a1d13-4350-4fdc-9852-8c6967b33dcd.xml" # "https://github.com/mrchapagain/food_ontology/blob/main/gs1_data/testdata.xml"

In [51]:
tree = ET.parse(path_tradesync_data)
root = tree.getroot()
print("Roots:", root.tag)
print("Attrib:", root.attrib)
print("Childerns: ", list(root))


Roots: {urn:gs1:gdsn:catalogue_item_notification:xsd:3}catalogueItemNotificationMessage
Attrib: {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'urn:gs1:gdsn:catalogue_item_notification:xsd:3 http://www.gdsregistry.org/3.1/schemas/gs1/gdsn/CatalogueItemNotification.xsd'}
Childerns:  [<Element '{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}StandardBusinessDocumentHeader' at 0x7f04610dc5f0>, <Element 'transaction' at 0x7f04610d42f0>]


***This part is hidden because the datast is very big and  take somany lines***

In [55]:
# Code to have a look whole data
"""# Reading the data inside the xml file to a variable under the name  data
with open(path_tradesync_data, 'r') as f:
    for line in f.readlines():
      print(line)
"""

# Code to explore tags
# The xml file has a LOT of tags!
list_of_items= []
stopAt = 200
for elem in root.iter():
    print(elem.tag)
    list_of_items.append(elem.tag)
    #print(elem.attrib)
    if stopAt==0:
        break
    stopAt -= 1

{urn:gs1:gdsn:catalogue_item_notification:xsd:3}catalogueItemNotificationMessage
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}StandardBusinessDocumentHeader
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}HeaderVersion
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Sender
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Identifier
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Receiver
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Identifier
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}DocumentIdentification
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}Standard
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}TypeVersion
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}InstanceIdentifier
{http://www.unece.org/cefact/namespaces/StandardBusinessDocumen

In [53]:
#print(ET.tostring(root, encoding='utf8').decode('utf8'))
list_of_items_to_find= ["entityIdentification", "brandName", "functionalName", "gln", "gtin", "creationDateTime", "dataRecipient", "contactDescription", "partyAddress", "gpcCategoryCode", "grossWeight", "descriptionShort", "allergenSpecificationName", "ingredientStatement", "tradeItemTemperatureConditionTypeCode", "uniformResourceIdentifier"]
#  "additionalTradeItemIdentification", "partyName",

#Lets start the dictionary to collecct items
dict_item_text = {}

# Loop over the tags that make sense (from list above)
for list_item in list_of_items_to_find:
  
  # Again  looping to get the text value
  for item in root.iter(list_item):
    dict_item_text[list_item] = item.text

# Lets see what we have c collected from the loop
dict_item_text

{'entityIdentification': 'CatalogueItemNotification-b10a17fc-ef9b-42b1-a0b1-76857c8a26b9',
 'brandName': 'Gammeldags Flødeis',
 'functionalName': 'Fløde is',
 'gln': '5790000701247',
 'gtin': '05766632526800',
 'creationDateTime': '2019-11-05T11:35:49.3113569+01:00',
 'dataRecipient': '5790000050000',
 'contactDescription': 'Premier Is, Mejerigaarden A/S',
 'partyAddress': 'Sennelsvej 1\n7700 Thisted',
 'gpcCategoryCode': '10000215',
 'grossWeight': '350',
 'descriptionShort': 'Karamel ',
 'allergenSpecificationName': '2003/89/EC',
 'ingredientStatement': 'Ingredienser: 35 % FLØDE, SKUMMETMÆLK, sukker, glukosesirup, sheaolie, rapsolie, sødet kondenseret SKUMMETMÆLK, SKUMMETMÆLKSPULVER, vand, kokosolie, fedtfattigt kakaopulver, Bourbon vaniljeekstrakt, ekstraherede vaniljekorn, SMØR, palmeolie, majsstivelse, helt hærdet kokosfedt, naturlig vaniljearoma, salt, aroma, paprikaekstrakt, emulgatorer (E 322 (af SOJA), E 471), stabilisatorer (E 466, E 412, E 407, E 440), surhedsregulerende mid

***Above dictinary are the information extracted from GS1 xml file associated with each item from the purchage list***

*We can grasp a lot of infoation about the item*